# Use of the static grid equivalents
This tutorial gives a quick overview of the grid equivalents function. This script gives an example to reduce a grid area using the IEEE case9 grid.

## Determination of grid areas

The user can determine the grid area of interest (internal area), the grid area for the reduction (external area), and the boundary buses between them. The figure below schows an example of the grid with identified grid areas:

- internal area (buses): [0, 3, 4, 8]
- boundary buses: [4, 8] (boundary buses belong to the internal area)
- external area (buses): [1, 2, 5, 6, 7] 

<img src="pics/grid areas.png" alt="ALT">

In order to identify the user-defined area, the boundary buses and one of the internal buses must be provided as inputs.


## Required inputs

The most important inputs for grid equivalents are:
- net: pandapower grid including power flow results
- eq_type: the method used to determine the equivalent grid, e.g., "rei", "ward", "xward"
- boundary_buses: indices of boundary buses (as a list) that divide the original grid into the internal areas and the external areas. Based on the figure above, boundary_buses = [4, 8] 
- internal_buses: bus indices (as a list), which are within the internal area. Based on the figure above, internal_buses = [0]. Just one of the internal buses is enough, the function will find and consider the remaining internal buses during the equivalent calculation. If 'internal_buses' is an empty list or None, the complete grid is treated as an external area.

## Example: REI-equivalent
In the following, the reduction of the external grid (buses [1, 2, 5, 6, 7] of the grid case 9) in the figure above is shown.
First, the necessary libraries need to be imported.

In [ ]:
import matplotlib.pyplot as plt

from pandapower.plotting.plotting_toolbox import set_line_geodata_from_bus_geodata
from pandapower.networks.power_system_test_cases import case9
from pandapower.toolbox.grid_modification import drop_lines
from pandapower.create import create_dcline
from pandapower.grid_equivalents import get_equivalent, get_boundaries_by_bus_zone_with_boundary_branches, set_bus_zone_by_boundary_branches
from pandapower.run import runpp

Subsequently, the IEEE case 9 grid model is created, and its power flow is calculated.

In [ ]:
net = case9()
net.sn_mva = 1.0
runpp(net)
net

In [ ]:
# equivalent type
eq_type = "rei"

# boundary buses
boundary_buses = [4, 8]

# internal buses 
internal_buses = [0]

Next, the equivalent function is called, and the grid reduction is executed.

In [ ]:
net_eq = get_equivalent(net, eq_type, boundary_buses, internal_buses)

Now, the grid equivalent is obtained: "net_eq" is the REI-equivalent grid model. We can compare the power flow results between the original grid "net" and the reduced grid "net_eq".  

In [ ]:
print("--- power flow (original grid) ---")
display(net.res_bus)
print("--- power flow (reduced grid) ---")
display(net_eq.res_bus)

It can be seen that the power flow results (*vm_pu*, *va_degree*) of the internal buses [0, 3, 4, 8] in both grids are the same , i.e., the equivalent calculation is successful. The *p_mw* and *q_mvar* values at the boundary buses [4, 8] are different due to the equivalent devices. During the grid reduction, additional equivalent devices (e.g., additional buses [9, 10, 11], impedance, shunts, etc.) are created, representing the interaction between the internal area and the external area, and maintaining the grid states in the internal area. We can see the additional shunts in the grid equivalent:

In [ ]:
print("--- shunts (original grid) ---")
display(net.shunt)
print("--- shunts (reduced grid) ---")
display(net_eq.shunt)

## Example: (X)Ward-equivalent

In the following, we demonstrate an example for the (x)ward-equivalent. We change the equivalent type and repeat the equivalent calculation.

In [ ]:
# equivalent type
eq_type = "ward"   # for xward-equivalent: eq_type = "xward"

# run equivalent calculation
net_eq_ward = get_equivalent(net, eq_type, boundary_buses, internal_buses)

Now, the ward equivalent is obtained. We can verify the power flow results.

In [ ]:
print("--- power flow (original grid) ---")
display(net.res_bus)
print("--- power flow (reduced grid) ---")
display(net_eq_ward.res_bus)

The power flow results (*vm_pu*, *va_degree*) for the internal buses [0, 3, 4, 8] in both grids are the same. The external area is represented by the addtional *ward* elements in the grid model "net_eq_ward", attached at the boundary buses [4, 8]: 

In [ ]:
print("--- ward (original grid) ---")
display(net.ward)
print("--- ward (reduced grid) ---")
display(net_eq_ward.ward)

# Defining the zones based on boundary branches

For this example we will use the lines between the buses 4 ... 5 and 7 ... 8

With the function *set_bus_zone_by_boundary_branches* we first define the zones of the grid based on this separation. Thereafter, with the function *get_boundaries_by_bus_zone_with_boundary_branches*, we obtain the boundary buses that we can use with the functions for obtaining the grid equivalents, as showed in previous sections.

In [ ]:
boundary_branches = {"line": [2, 7]}
set_bus_zone_by_boundary_branches(net, boundary_branches)
buses, branches = get_boundaries_by_bus_zone_with_boundary_branches(net)

The dictionary *buses* contains the biundary buses, internal buses and external buses from the point of view of each of the zones.

In [ ]:
buses

The resulting separation can be seen in the figure below. The green buses are the nodes of the boundary lines that are part of the internal zone. The red buses are the nodes of the boundary lines that are in the external zone. The red dashed lines are the boundary lines.

In [ ]:
zone = 1

set_line_geodata_from_bus_geodata(net, overwrite=True)

import geopandas as gpd
import geojson

fig, ax=plt.subplots(figsize=(7,11))

net.bus.geo = net.bus.geo.apply(lambda x: geojson.loads(x))
net.bus = gpd.GeoDataFrame(net.bus, geometry='geo')
net.line.geo = net.line.geo.apply(lambda x: geojson.loads(x))
net.line = gpd.GeoDataFrame(net.line, geometry='geo')

net.line.plot(ax=ax)
net.line.loc[list(boundary_branches["line"])].plot(ax=ax, color='red', linestyle="--")
net.bus.plot(column=net.bus.zone.values, ax=ax,zorder=10)
net.bus.loc[net.ext_grid.bus.values].plot(ax=ax, color='k', marker="s", markersize=200)

net.bus.loc[list(buses[zone]["external"])].plot(color='r', ax=ax, zorder=100, markersize=100)
net.bus.loc[list(buses[zone]["internal"])].plot(color='g', ax=ax, zorder=100, markersize=100)

Now, we can obtain the grid equivalent to reduce the zone 0

In [ ]:
b_internal = [net.bus.loc[net.bus.zone==1].index[0]]
net_eq = get_equivalent(net, "rei", buses[1]["internal"], b_internal)

In [ ]:
net_eq.bus

## Grid equivalents with DC lines

In [ ]:
drop_lines(net, [2])
create_dcline(net, 4, 5, 50, 0.5, 0.7, 1.0, 1.0)
runpp(net)

In [ ]:
net_eq = get_equivalent(net, "rei", [4,8], [0])

In [ ]:
net_eq.bus